In [27]:
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Reshape, Lambda, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, UpSampling2D,Conv2D, Convolution1D
from keras.optimizers import SGD, Adam
from keras.layers.normalization import BatchNormalization
from keras.layers import merge, Input
from keras.callbacks import ModelCheckpoint, Callback, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras import backend as K
from keras.regularizers import l2
from theano.sandbox.cuda import CudaNdarray
from copy import deepcopy
import numpy as np
from scipy.misc import imsave, imshow, imread
import h5py
import matplotlib.pyplot as plt
from keras.utils import np_utils
#from np_utils import to_categorical


In [2]:
model = ResNet50(include_top=False, weights='imagenet',input_tensor=Input(shape=(3,240,320)))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 240, 320)   0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 246, 326)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 120, 160)  9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 120, 160)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [35]:
from copy import deepcopy
import time

def DataGenerator(batchsize=5):
    #infinite loop to provide for the samples..
    len1 = 56
    while 1:
        file1 = h5py.File('newtrain.h5')
        xtrain = np.array(file1['xtrain1'], dtype=np.float32)
            #ytrain = np.array(file1['ytrain1'], dtype=np.float32)
            
        len2 = xtrain.shape[0]
        batches = len2/batchsize
        cal1 = batches * batchsize
        xtrain = xtrain[:cal1]
            #ytrain1 = ytrain1[:cal1]
        for ind in range(0,batches):
                
            xtrain1 = deepcopy(xtrain[ind*batchsize:ind*batchsize+batchsize,:,:])
            ytrain1 = deepcopy(big[ind*batchsize:ind*batchsize+batchsize,:,:])
                
            xtrain1 /= 255
                
            gpu_xtrain1 = CudaNdarray(xtrain1)
            gpu_ytrain1 = CudaNdarray(ytrain1)
                
            yield gpu_xtrain1,gpu_ytrain1
            time.sleep(0.1)

In [6]:
out1 = model.layers[174].output

temp1 = UpSampling2D(size=(60,80))(out1) 
temp1 = Convolution2D(4800,1,1,activation='relu')(temp1)
temp1 = Dropout(0.5)(temp1)
temp1 = Convolution2D(50,1,1,activation='linear')(temp1)
temp1 = Reshape((50,4800))(temp1)
#temp1 = Convolution2D(4800,1,1)(temp1)
temp1 = Activation('softmax')(temp1)
#temp1 = Reshape((50,60,80))(temp1)
#temp1 = Convolution2D(1,1,1,activation='relu')(temp1)
#temp1 = Reshape((1,4800))(temp1)


model1 = Model(input=model.layers[0].input, output=temp1)
model1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 240, 320)   0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 246, 326)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 120, 160)  9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 120, 160)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
v1 = model1.predict(np.random.rand(2,3,240,320))
v1.shape

In [19]:
sgd = SGD(lr=0.001,momentum=0.9)
model1.compile(loss='sparse_categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
#model1.compile(loss=catentropy,optimizer=sgd,metrics=['accuracy'])

In [8]:
K.image_dim_ordering()

'th'

In [3]:
#dataset loading part... needs to be in channel_last format as the model definition is in that format...
#tar1 = "/media/saurav/E20A35060A34D96D/WRNTry1/functional-zoo-master/"
file1 = h5py.File('newtrain.h5')
xtrain = np.array(file1['xtrain1'], dtype=np.float32)
ytrain = np.array(file1['ytrain1'], dtype=np.float32)
print(xtrain.shape)
print(ytrain.shape)

(795, 3, 240, 320)
(795, 60, 80)


In [10]:
ytrain1 = deepcopy(ytrain)
print(ytrain1[0])

bins = np.arange(0,10.2,0.2)
print(bins)

binytrain = np.digitize(ytrain1,bins)
print("shape ---> ",binytrain.shape)
print(binytrain[0])

[[ 2.14767098  2.14495873  2.14624715 ...,  1.58545399  1.58290648
   1.57842386]
 [ 2.14393187  2.13483667  2.13384104 ...,  1.58497679  1.5843972
   1.57729566]
 [ 2.14251614  2.13496757  2.13421893 ...,  1.57982635  1.57913637
   1.57486784]
 ..., 
 [ 1.67584753  1.62338197  1.60583317 ...,  1.37740159  1.37847006
   1.46694279]
 [ 1.66630363  1.62306476  1.59586525 ...,  1.38042998  1.38017082
   1.46754074]
 [ 1.6628015   1.64124727  1.62007165 ...,  1.44635117  1.44452393
   1.47627556]]
[  0.    0.2   0.4   0.6   0.8   1.    1.2   1.4   1.6   1.8   2.    2.2
   2.4   2.6   2.8   3.    3.2   3.4   3.6   3.8   4.    4.2   4.4   4.6
   4.8   5.    5.2   5.4   5.6   5.8   6.    6.2   6.4   6.6   6.8   7.
   7.2   7.4   7.6   7.8   8.    8.2   8.4   8.6   8.8   9.    9.2   9.4
   9.6   9.8  10. ]
('shape ---> ', (795, 60, 80))
[[11 11 11 ...,  8  8  8]
 [11 11 11 ...,  8  8  8]
 [11 11 11 ...,  8  8  8]
 ..., 
 [ 9  9  9 ...,  7  7  8]
 [ 9  9  8 ...,  7  7  8]
 [ 9  9  9 ...,  8  8 

In [11]:
#changes to the range 0to49....
binytrain -= 1
print("min ",np.min(binytrain))
print("max ",np.max(binytrain))

('min ', 3)
('max ', 49)


In [12]:
label = np.reshape(binytrain,(795,4800))
print("shape label --> ",label.shape)
print("binytrain shape ---> ",binytrain.shape)

('shape label --> ', (795, 4800))
('binytrain shape ---> ', (795, 60, 80))


In [ ]:
def catentropy(y_true,y_pred):
    return K.mean(K.categorical_crossentropy(y_pred, K.cast(y_true.flatten(),dtype='int32')), axis=-1)


In [ ]:
#out1 = model1.predict(np.random.rand(2,3,240,320))
#out1.shape
#xtrain.shape
#print(n1.shape)
big.shape

In [36]:
#check1 = model1.fit(xtrain,big,nb_epoch=1, batch_size=2, verbose=1, validation_split=0.05)
epochs=1
history2 = model1.fit_generator(DataGenerator(5),samples_per_epoch=795, nb_epoch=epochs)

Epoch 1/1


ValueError: Error when checking model target: expected activation_50 to have shape (None, 50, 1) but got array with shape (5, 50, 4800)

In [ ]:
n1 = np.random.rand(795,51,1)
n1.shape
#c1 = binytrain[0]
#c1.shape
#s1 = np.reshape(c1,(1,-1))
#print(s1.shape)

In [32]:
#np_utils.to_categorical()
#binytrain.shape
#label.shape
big.dtype

dtype('float32')

In [ ]:
#np.min(binytrain)
t = range(1,51)
t1 = np_utils.to_categorical(t)
t1.shape

In [ ]:
print("label shape is ---> ",label.shape)
fc1 = np_utils.to_categorical(label[100])
print(fc1.shape)
print(fc1[0])
print(np.where(fc1[0] == 1))

In [14]:
big = np.zeros((795,50,4800))
print("big shape   --> ",big.shape)
print("label shape --> ",label.shape)

('big shape   --> ', (795, 50, 4800))
('label shape --> ', (795, 4800))


In [15]:
for i in range(795):
    for j in range(4800):
        sample1 = label[i][j]
        #print("sample1 value --> ",sample1)
        #print("sample1 shape --> ",sample1.shape)
        ind = np_utils.to_categorical(sample1)
        #print("ind is --> ",ind)
        loc = np.where(ind == 1)
        #print("loc  ---> ",loc[1][0])
        big[i][loc[1][0]][j] = 1
               

In [16]:
big.shape

(795, 50, 4800)

In [ ]:
big1 = np.zeros((795,4800,51))

In [ ]:
for i in range(795):
    for j in range(4800):
        sample1 = label[i][j]
        #print("sample1 value --> ",sample1)
        #print("sample1 shape --> ",sample1.shape)
        ind = np_utils.to_categorical(sample1)
        #print("ind is --> ",ind)
        loc = np.where(ind == 1)
        #print("loc  ---> ",loc[1][0])
        big1[i][j][loc[1][0]] = 1

In [ ]:
print(label[0][1])
print(big1[0][1])

In [ ]:
s1 = np_utils.to_categorical(label)

In [ ]:
s1.shape